In [26]:
import pandas
import numpy
import scipy
import string
import numpy as np
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [21]:
file = open("../marktwain/treated/combined.txt", "r")
lines = file.readlines()
filestring = " ".join(lines)


In [23]:
%store filestring

Stored 'filestring' (str)


In [29]:
stringArray = filestring.split(" ")
chunksArr = np.array_split(stringArray, 10000)
len(chunksArr[0])

54

In [59]:
mtArray = []
for item in chunksArr:
    global mtArray
    item = [item.tolist(),['marktwain']]
    mtArray.append(item)
%store mtArray

sentenceArray = []
for item in mtArray:
    sentence = doc2vec.LabeledSentence(
    words=item[0], tags=item[1])
    sentenceArray.append(sentence)
sentenceArray

Stored 'mtArray' (list)


[LabeledSentence(words=['a', 'connecticut', 'yankee', 'in', 'king', 'arthurs', 'court', 'a', 'word', 'of', 'explanation', 'it', 'was', 'in', 'warwick', 'castle', 'that', 'i', 'came', 'across', 'the', 'curious', 'stranger', 'whom', 'i', 'am', 'going', 'to', 'talk', 'about', 'he', 'attracted', 'me', 'by', 'three', 'things', 'his', 'candid', 'simplicity', 'his', 'marvelous', 'familiarity', 'with', 'ancient', 'armor', 'and', 'the', 'restfulness', 'of', 'his', 'company', 'for', 'he', 'did'], tags=['marktwain']),
 LabeledSentence(words=['all', 'the', 'talking', 'we', 'fell', 'together', 'as', 'modest', 'people', 'will', 'in', 'the', 'tail', 'of', 'the', 'herd', 'that', 'was', 'being', 'shown', 'through', 'and', 'he', 'at', 'once', 'began', 'to', 'say', 'things', 'which', 'interested', 'me', 'as', 'he', 'talked', 'along', 'softly', 'pleasantly', 'flowingly', 'he', 'seemed', 'to', 'drift', 'away', 'imperceptibly', 'out', 'of', 'this', 'world', 'and', 'time', 'and', 'into', 'some'], tags=['mark

In [71]:
from gensim.models import doc2vec
model = doc2vec.Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
model.build_vocab(sentenceArray)


In [81]:
for epoch in range(10):
    model.train(sentenceArray, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

In [85]:
# store the model to mmap-able files
model.save('/tmp/marktwain_model.doc2vec')
# load the model back
model_loaded = doc2vec.Doc2Vec.load('/tmp/marktwain_model.doc2vec')

In [90]:
model.most_similar('good')

[('deal', 0.5960935354232788),
 ('reception', 0.5532675981521606),
 ('circulation', 0.5335501432418823),
 ('parlous', 0.510429322719574),
 ('brotherhood', 0.5074785947799683),
 ('engagement', 0.49928784370422363),
 ('hearty', 0.4978508949279785),
 ('courtship', 0.487670361995697),
 ('meiner', 0.48674696683883667),
 ('friendsd', 0.4862056076526642)]

In [92]:
model['bad']

array([-1.89936435, -2.2803669 ,  1.68926287,  1.9192915 ,  0.04442691,
        2.53106713, -1.74775207, -1.30237758,  1.90330446, -0.65056252,
       -1.33033347, -1.6238445 ,  1.29951012, -0.39143041, -0.89531785,
        3.09691072,  1.48653889, -1.32385111,  0.21125172, -0.80604607,
        0.39908931,  0.22957164, -1.00876415,  1.33571041,  0.13517706,
        2.33368516, -0.18365517, -1.57213247, -1.8735044 , -1.14467728,
        3.02335572,  1.70979893, -1.37104154, -0.5976634 ,  2.60021734,
       -0.28548983, -0.05941117,  0.48461065, -1.71820068, -1.9099108 ,
       -1.79717171, -0.00459263,  0.82517815, -1.68004918,  1.1580739 ,
        0.53923661,  0.63749361,  2.85535741,  0.6431523 ,  0.13723744,
       -0.69905144, -0.30946735, -0.57365209, -2.56070304, -1.28075266,
        0.81961733,  0.06121459, -1.32932639,  1.10976493,  0.62147874,
        0.70552492, -0.69225448, -2.42534423,  0.02900346, -0.70324832,
       -2.75659037,  0.90390974,  1.64259958, -0.61268163, -0.44